<a href="https://colab.research.google.com/github/t3manish/Build-chat-applications-with-openai-and-langchain/blob/main/365_Q_AND_A_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install langchain==0.3.3 langchain-chroma==0.1.4 langchain-community==0.3.2 langchain-openai==0.2.2 pypdf==5.0.1 python-dotenv==1.0.1

In [1]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# You can add this line to check that it loaded (optional)
# print(f"API Key Loaded: {os.environ['OPENAI_API_KEY'][:5]}...")

In [3]:
from langchain_community.document_loaders.pdf import PyPDFLoader

from langchain_text_splitters import (MarkdownHeaderTextSplitter,
                                      TokenTextSplitter)

from langchain_core.output_parsers.string import StrOutputParser
from langchain_core.messages import SystemMessage
from langchain_core.prompts import (PromptTemplate,
                                    HumanMessagePromptTemplate,
                                    ChatPromptTemplate)
from langchain_core.runnables import (RunnablePassthrough,
                                      RunnableLambda,
                                      RunnableParallel, # Added this line
                                      chain)

from langchain_openai import (ChatOpenAI,
                              OpenAIEmbeddings)

from langchain_chroma.vectorstores import Chroma

In [4]:
from google.colab import files

uploaded = files.upload()

print(f"Uploaded '{list(uploaded.keys())[0]}' to Colab.")

Saving 365_Data_Science_Courses.pdf to 365_Data_Science_Courses (1).pdf
Uploaded '365_Data_Science_Courses (1).pdf' to Colab.


In [5]:
from langchain_community.document_loaders import PyPDFLoader

# Define the file path (it's the name of the file you uploaded)
file_path = "365_Data_Science_Courses.pdf"

# Create an instance of the loader
loader = PyPDFLoader(file_path)

# Load the documents
pages_pdf = loader.load()

# Print the length of the list
print(f"The number of pages (documents) loaded is: {len(pages_pdf)}")

The number of pages (documents) loaded is: 3


In [6]:
!pip install langchain-community pypdf
from langchain_community.document_loaders import PyPDFLoader
import copy

In [7]:
from langchain_community.document_loaders import PyPDFLoader

# Load the PDF
loader = PyPDFLoader("365_Data_Science_Courses.pdf")
pages_pdf = loader.load()

print(f"--- Q7 Answer ---")
print(f"Length of pages_pdf list: {len(pages_pdf)}")

--- Q7 Answer ---
Length of pages_pdf list: 3


In [8]:
from langchain.text_splitter import MarkdownHeaderTextSplitter

# Join all page content into one string
pages_string = "\n".join([doc.page_content for doc in pages_pdf])

# Define the headers to split on
headers_to_split_on = [
    ("#", "Course Title"),
    ("##", "Lecture Title")
]

# Create the splitter
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on
)

# Split the text
pages_md_split = markdown_splitter.split_text(pages_string)

print(f"--- Q8 Answer ---")
print(f"Length of pages_md_split list: {len(pages_md_split)}")

--- Q8 Answer ---
Length of pages_md_split list: 4


In [9]:
from langchain.text_splitter import CharacterTextSplitter

# Create the splitter
char_splitter = CharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=0,
    separator="\n" # The exam specifies no overlap
)

# Split the documents
pages_char_split = char_splitter.split_documents(pages_md_split)

print(f"--- Q9 Answer ---")
print(f"Length of pages_char_split list: {len(pages_char_split)}")

--- Q9 Answer ---
Length of pages_char_split list: 17


In [13]:
import numpy as np
from langchain_openai import OpenAIEmbeddings

# Initialize the embedding model
embedding = OpenAIEmbeddings(model="text-embedding-ada-002")

# Get the content of the two chunks
content_7 = pages_char_split[1].page_content
content_8 = pages_char_split[8].page_content

# Embed the two chunks
vec_7 = embedding.embed_query(content_7)
vec_8 = embedding.embed_query(content_8)

# Calculate the dot product
dot_product = np.dot(vec_7, vec_8)

print(f"--- Q10 Answer ---")
print(f"Dot product: {dot_product:.2f}")

--- Q10 Answer ---
Dot product: 0.77


In [11]:
from langchain_community.vectorstores import Chroma

# Create the vector store and embed the documents
vector_store = Chroma.from_documents(
    documents=pages_char_split,
    embedding=embedding,
    persist_directory="365-ds-courses"
)

# Initialize the retriever
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 3, 'lambda_mult': 0.7}
)

# Test the retriever
test_query = "Could you give me some examples of big data used in real life?"
retrieved_docs = retriever.invoke(test_query)

print(f"--- Q11 Test ---")
print("Retrieved documents contain metadata from different lectures:")
for i, doc in enumerate(retrieved_docs):
    print(f"Doc {i+1}: {doc.metadata['Lecture Title']}")

print("\nThis confirms Answer 4 ('fetches from the same lecture') is incorrect.")

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


--- Q11 Test ---
Retrieved documents contain metadata from different lectures:
Doc 1: Big Data Real -life Examples
Doc 2: Big Data Real -life Examples
Doc 3: Machine Learning (ML) Real -life Examples

This confirms Answer 4 ('fetches from the same lecture') is incorrect.


In [12]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

# --- Q12: Define Prompts ---
# Load the system prompt from the file
with open("Message_Template_1.txt", "r") as f:
    message_template_1 = f.read()

# Create the human prompt
message_template_2 = """
Resources:
{context}

User Question:
{question}
"""

# Combine into a ChatPromptTemplate
chat_template = ChatPromptTemplate.from_messages([
    ("system", message_template_1),
    ("human", message_template_2)
])

# --- Q13 & Q14: Create the parallel chain ---
# This RunnableParallel dictionary defines the inputs for the prompt
# It runs the retriever and the passthrough in parallel.
chain_map = RunnableParallel(
    {
        "context": retriever,
        "question": RunnablePassthrough() # Q13 Answer
    }
) # Q14 Answer: We wrap the dict in RunnableParallel

# --- Q15, Q16, Q17: Build and invoke the full chain ---
# Define the LLM
chat = ChatOpenAI(
    model="gpt-4",
    temperature=0,
    seed=365,
    max_tokens=250
)

# Define the output parser
str_parser = StrOutputParser()

# Build the full chain
final_chain = chain_map | chat_template | chat | str_parser

# --- Invoke the chain ---
user_question = "Could you give me some examples of big data used in real life?"
response = final_chain.invoke(user_question)

print(f"\n--- Q17: Final RAG Response ---")
print(response)


--- Q17: Final RAG Response ---
Sure, big data is used in various industries and companies. One notable example is Facebook, one of the largest online communities. Facebook keeps track of its users' names, personal data, photos, videos, recorded messages, and so on. This means their data has a lot of variety and can be considered as 'big data'.

Resources: 
1. Lecture "Big Data Real -life Examples" from the "Introduction to Data and Data Science" course
